In [1]:
# relative MTADelayPredict Project
import sys
import os
import pandas as pd

sys.path.append(os.path.dirname(os.path.dirname(os.path.realpath(os.path.join('DataExploration.ipynb')))))
from MTADelayPredict.utils import gtfs_loader, stop_info
from MTADelayPredict.subway_line import SubwayLine, N_STOP_LIST
from MTADelayPredict.stop import Stop
from importlib import reload
from MTADelayPredict.plotting import alerts

In [2]:
import re

subway_line = SubwayLine(N_STOP_LIST)
obs_stop = subway_line.stop('R16N')

def stop_processor(name):
    name = str(name)
    if str.lower(name) == 'nan':
        return ''
    
    # Strip parenthesis borough delimeters, we know what line we're on
    stop_ids = stop_info.name2stop_ids(name, N_STOP_LIST)
    
    # If there were issues parsing, check a few things
    if len(stop_ids) == 0:
        # Is this a range? 
        range_m = re.match(r'^(.+),(.+)', name)
        if range_m:
            stop_1 = subway_line.stop(stop_info.name2stop_ids(range_m.groups()[0], N_STOP_LIST).iloc[0])
            stop_2 = subway_line.stop(stop_info.name2stop_ids(range_m.groups()[1], N_STOP_LIST).iloc[0])
            
            if abs(stop_1.stop_idx - obs_stop.stop_idx) < abs(stop_2.stop_idx - obs_stop.stop_idx):
                return stop_1.stop_id
            else:
                return stop_2.stop_id
            
        # Still unparsable
        return name
        
    return stop_ids.iloc[0]

In [3]:
import pandas as pd
import numpy as np
import os 

alert_dir = '../data/raw/alerts'
annotated_alert_df = pd.read_csv(os.path.abspath(os.path.join(alert_dir, 'nqrw_alerts.csv')))

n_alert_df = annotated_alert_df[annotated_alert_df['Direction'] == 'Northbound']
n_alert_df['IssueStopID'] = n_alert_df['IssueStop'].map(stop_processor)

/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [4]:
import plotly.graph_objects as go
import plotly.express as px

def create_figure(schedule_df, alert_stop, observing_stop, date, start_time, end_time):
    traces = []

    subway_line = SubwayLine(N_STOP_LIST)

    for col in schedule_df.columns[2:]:
        plot_df = schedule_df[['index', col]].dropna()
        if (plot_df.shape[0] == 0 or plot_df['index'].iloc[-1] < start_time) or (plot_df['index'].iloc[0] > end_time):
            continue

        t = go.Scatter(
                    x= plot_df['index'],
                    y = plot_df[col],
                    mode='lines+markers',
                    name=col,
                    x0=schedule_df['index'].iloc[0]
            )

        traces.append(t)
    shapes = list()

    if alert_stop:
        # Alert time
        shapes.append({'type': 'line',
                       'xref': 'x',
                       'yref': 'y',
                       'x0': date,
                       'x1': date,
                       'y0': 0,
                       'y1': schedule_df.iloc[:, 2:].max().max(),
                       'line_color': 'red'
                      })

        # Alert stop
        shapes.append({'type': 'line',
                       'xref': 'x',
                       'yref': 'y',
                       'x0': schedule_df['index'].min(),
                       'x1': schedule_df['index'].max(),
                       'y0': subway_line.stop_idx(alert_stop),
                       'y1': subway_line.stop_idx(alert_stop),
                       'line_color': 'red'
                      })

    # Observed stop
    shapes.append({'type': 'line',
                   'xref': 'x',
                   'yref': 'y',
                   'x0': schedule_df['index'].min(),
                   'x1': schedule_df['index'].max(),
                   'y0': subway_line.stop_idx(observing_stop),
                   'y1': subway_line.stop_idx(observing_stop),
                   'line_color': 'green'
                  })

    layout = go.Layout(
                title= ('N Train Subways @ {}'.format(date)),
                titlefont=dict(
                family='Courier New, monospace',
                size=15,
                color='#7f7f7f'
                ),
                paper_bgcolor='rgba(0,0,0,0)',
                plot_bgcolor='rgba(0,0,0,0)',
                width=1500,
                height=800,

                xaxis=dict(
                    tickmode = 'array',
                    tickvals = schedule_df['index'],
                    ticktext = schedule_df['index'],
                ),
                yaxis=dict(
                    tickmode = 'array',
                    tickvals = [i for i,_ in enumerate(N_STOP_LIST)],
                    ticktext = [stop_info.stop_id2name(s) for s in N_STOP_LIST],
                    title='Stop',
                ),
                shapes=shapes
    )
    return {'data':traces, 'layout':layout}


In [8]:
from datetime import datetime as dt
import dash
from dash.dependencies import Input, Output
import dash_html_components as html
import dash_core_components as dcc
import dash_table
import re
from dash.exceptions import PreventUpdate

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']


app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.layout = html.Div([
    dcc.DatePickerSingle(
        id='schedule-date',
        min_date_allowed=dt(2018, 8, 1),
        max_date_allowed=dt(2019, 6, 30),
        initial_visible_month=dt(2018, 8, 1),
        date=str(dt(2018, 8, 1))
    ),
    html.Button('Update', id='update-button', n_clicks=0),
#    html.Button('Next', id='next-button', n_clicks=0),
    html.Div(id='current-date'),
    dcc.Graph(id='alert-graph'),
    # Hidden div to keep track of what idx we're on
    html.Div(id='date-val', style={'display': 'none'})
])


@app.callback(
    Output('current-date', 'children'),
    [Input('schedule-date', 'date')])
def update_output(date):
    string_prefix = 'Data Range: '

    ts = pd.Timestamp(date).tz_localize("US/Eastern")
    start_time = ts.replace(hour=0, minute=0, second=0)
    end_time = ts.replace(hour=23, minute=59, second=59)
    
    return string_prefix + str(start_time) + ' - ' + str(end_time)

@app.callback(
    Output('alert-graph', 'figure'),
[Input('schedule-date', 'date'), Input('update-button', 'n_clicks')])
def update_plot(date, n_clicks):
    ctx = dash.callback_context
    
    if not ctx.triggered:
        print("No Trigger")
        raise PreventUpdate
    else:
        trigger_id = ctx.triggered[0]['prop_id'].split('.')[0]
        
    print("Triggered by {}".format(trigger_id))
    if trigger_id == 'schedule-date':
        raise PreventUpdate
    
    start_window = 60
    end_window = 60
    OBSERVED_STOP = 'R16N'
    
    ts = pd.Timestamp(date).tz_localize("US/Eastern")

    STOP_FILTER = '^.*N$'
    ROUTE_FILTER = 'N'
    data_dir = '../data/raw/status'
    
    start_time = ts.replace(hour=0, minute=0, second=0)
    end_time = ts.replace(hour=23, minute=59, second=59)

    # Fetch schedule data and plot
    schedule_df = alerts.load_range_schedule(start_time, end_time, STOP_FILTER, ROUTE_FILTER, data_dir)['schedule_df']
    
    fig_dict =  create_figure(schedule_df, None, OBSERVED_STOP, ts, start_time, end_time)
#    fig = go.Figure(fig_dict)
#    fig.show()
    return fig_dict
    
app.run_server(host='0.0.0.0', port='8050', debug=True, use_reloader=False)  # Turn off reloader if inside Jupyter

Running on http://0.0.0.0:8050/
Running on http://0.0.0.0:8050/
Running on http://0.0.0.0:8050/
Running on http://0.0.0.0:8050/
Debugger PIN: 448-375-672
Debugger PIN: 448-375-672
Debugger PIN: 448-375-672
Debugger PIN: 448-375-672
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
No Trigger
Triggered by schedule-date
Triggered by schedule-date
Triggered by update-button
25640879
25640999


100%|#####################################|entries: ------decode_errors:     53


New stops:
set()
